In [9]:
from keras.datasets import boston_housing
from keras import models, layers
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [38]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

print(train_data.shape, test_data.shape)
print(train_data[1])
import pandas as pd
dd = pd.DataFrame(train_data)
print(train_targets[1])

print(dd)
dd

(404, 13) (102, 13)
[2.1770e-02 8.2500e+01 2.0300e+00 0.0000e+00 4.1500e-01 7.6100e+00
 1.5700e+01 6.2700e+00 2.0000e+00 3.4800e+02 1.4700e+01 3.9538e+02
 3.1100e+00]
42.3
0       1.23247
1       0.00000
2       8.14000
3       0.00000
4       0.53800
5       6.14200
6      91.70000
7       3.97690
8       4.00000
9     307.00000
10     21.00000
11    396.90000
12     18.72000
Name: 0, dtype: float64


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.23247,0.0,8.14,0.0,0.5380,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.4150,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.6310,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.5150,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.7130,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.21977,0.0,6.91,0.0,0.4480,5.602,62.0,6.0877,3.0,233.0,17.9,396.90,16.20
400,0.16211,20.0,6.96,0.0,0.4640,6.240,16.3,4.4290,3.0,223.0,18.6,396.90,6.59
401,0.03466,35.0,6.06,0.0,0.4379,6.031,23.3,6.6407,1.0,304.0,16.9,362.25,7.83
402,2.14918,0.0,19.58,0.0,0.8710,5.709,98.5,1.6232,5.0,403.0,14.7,261.95,15.79


In [11]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [12]:
def build_model():
    model = models.Sequential()
    model.add(layers.Input(shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(63, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [13]:
def mean_absolute_percentage_error(true, pred):
    abs_percentage_error = []
    for true_val, pred_val in zip(true, pred):
        abs_error = np.abs(true_val - pred_val)
        abs_percentage_error.append(abs_error / max(np.abs(true_val), 1))
    mape = np.mean(abs_percentage_error) * 100
    return mape

In [17]:
import math
model = build_model()

model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

prediction = model.predict(train_data)
test_mae_score = mean_absolute_error(train_targets, prediction)
test_mape_score = mean_absolute_percentage_error(train_targets, prediction)
mse = mean_squared_error(train_targets, prediction)
test_rmse_score = math.sqrt(mse)

print("-"*50)
print("MAE score:", test_mae_score)
print("MAPE score:", test_mape_score)
print("RMSE score:", test_rmse_score)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 21.3298 - mae: 2.3319 
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
--------------------------------------------------
MAE score: 1.7301583811788277
MAPE score: 8.910972625017166
RMSE score: 2.3833867083840534


In [21]:
prediction = model.predict(test_data)

mae = mean_absolute_error(test_targets, prediction)
mape = mean_absolute_percentage_error(test_targets, prediction)
mse = mean_squared_error(test_targets, prediction)
rmse = math.sqrt(mse)

test_mape_score = mean_absolute_percentage_error(test_targets, prediction)

print("-"*50)
print("MAE score:", mae)
print("MAPE score:", mape)
print("RMSE score:", rmse)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
--------------------------------------------------
MAE score: 2.5277563244688745
MAPE score: 13.188786804676056
RMSE score: 4.135281860983755


**超參數實驗**

In [23]:
from keras import backend as K

def create_model(input_size, layer_number, unites_number, activation, optimizer):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_size,)))
    for i in range(layer_number):
        model.add(layers.Dense(unites_number, activation=activation))
    model.add(layers.Dense(1))
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

layers_to_test = [1, 2, 3]
unites_number = [64, 128, 256, 512, 1024]
epochs_to_test = [5, 10, 20]
batch_size = [32, 64, 128, 256]

all_record = []

activation = 'relu'
optimizer = 'adam'
input_size = train_data.shape[1]

for lay in layers_to_test:
    for unit in unites_number:
        for epo in epochs_to_test:
            for bat in batch_size:
                test_model = create_model(input_size, lay, unit, activation, optimizer)
                history = test_model.fit(train_data, train_targets, epochs=epo, batch_size=bat, verbose=2)
                train_mse_score, train_mae_score = test_model.evaluate(train_data, train_targets)
                
                train_predict = test_model.predict(train_data)
                train_mape_score = mean_absolute_percentage_error(train_targets, train_predict)
                train_rmse_score = math.sqrt(train_mse_score)
                
                print("-"*50)
                print(f"隱藏層{lay}, 節點{unit}, epoch{epo}, batch{bat} 訓練集績效:")
                print("MAE score:", train_mae_score)
                print("MAPE score:", train_mape_score)
                print("RMSE score:", train_rmse_score)

                test_prediction = test_model.predict(test_data)      
                test_mape_score = mean_absolute_percentage_error(test_targets, test_prediction)
                test_rmse_score = math.sqrt(test_mse_score)
                
                print("-"*50)
                print(f"隱藏層{lay}, 節點{unit}, epoch{epo}, batch{bat} 測試集績效:")
                print("MAE score:", test_mae_score)
                print("MAPE score:", test_mape_score)
                print("RMSE score:", test_rmse_score)

                current_result = {"layer": lay,
                                  "units": unit,
                                  "epochs": epo,
                                  "batch_size": bat,
                                  "activation": activation,
                                  "optimizer": optimizer,
                                  "train_mae": train_mae_score,
                                  "train_mape": train_mape_score,
                                  "train_rmse": train_rmse_score,
                                  "test_mae": test_mae_score,
                                  "test_mape": test_mape_score,
                                  "test_rmse": test_rmse_score
                                 }
                
                all_record.append(current_result)
                K.clear_session()
    print("="*50)
    print(f"Completed batch for layer {lay}")
    print("="*50)

Epoch 1/5
13/13 - 1s - 53ms/step - loss: 597.1650 - mae: 22.6978
Epoch 2/5
13/13 - 0s - 5ms/step - loss: 576.7426 - mae: 22.2594
Epoch 3/5
13/13 - 0s - 3ms/step - loss: 561.3062 - mae: 21.8222
Epoch 4/5
13/13 - 0s - 4ms/step - loss: 539.0442 - mae: 21.3758
Epoch 5/5
13/13 - 0s - 5ms/step - loss: 520.1869 - mae: 20.9032
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 574us/step - loss: 489.6769 - mae: 20.1663
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
--------------------------------------------------
隱藏層1, 節點64, epoch5, batch32 訓練集績效:
MAE score: 20.625083923339844
MAPE score: 90.53438901901245
RMSE score: 22.447683589518903
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
--------------------------------------------------
隱藏層1, 節點64, epoch5, batch32 測試集績效:
MAE score: 1.7301583811788277
MAPE score: 90.26145935058594
RMSE score: 6.315954792636419

Epoch 1/5
7/7 - 1s - 100ms/step - loss: 560.2032 - mae: 22.2083
Epoch 2/5
7/7 - 0s - 10ms/step - loss: 563.6415 - mae: 21.9651
Epoch 3/5
7/7 - 0s - 5ms/step - loss: 573.3764 

In [24]:
print(all_record)

[{'layer': 1, 'units': 64, 'epochs': 5, 'batch_size': 32, 'activation': 'relu', 'optimizer': 'adam', 'train_mae': 20.625083923339844, 'train_mape': 90.53438901901245, 'train_rmse': 22.447683589518903, 'test_mae': 1.7301583811788277, 'test_mape': 90.26145935058594, 'test_rmse': 6.315954792636419}, {'layer': 1, 'units': 64, 'epochs': 5, 'batch_size': 64, 'activation': 'relu', 'optimizer': 'adam', 'train_mae': 21.05048942565918, 'train_mape': 93.08838844299316, 'train_rmse': 22.813061850957776, 'test_mae': 1.7301583811788277, 'test_mape': 92.93878078460693, 'test_rmse': 6.315954792636419}, {'layer': 1, 'units': 64, 'epochs': 5, 'batch_size': 128, 'activation': 'relu', 'optimizer': 'adam', 'train_mae': 21.377948760986328, 'train_mape': 94.89657878875732, 'train_rmse': 23.095235278691685, 'test_mae': 1.7301583811788277, 'test_mape': 94.68661546707153, 'test_rmse': 6.315954792636419}, {'layer': 1, 'units': 64, 'epochs': 5, 'batch_size': 256, 'activation': 'relu', 'optimizer': 'adam', 'train_

In [26]:
import pandas as pd
record_data_forConvert = pd.DataFrame(all_record)
record_df = pd.DataFrame.from_records(record_data_forConvert)
record_df.to_csv('./lab2.csv', index=False)
record_df

,layer,units,epochs,batch_size,activation,optimizer,train_mae,train_mape,train_rmse,test_mae,test_mape,test_rmse
0,1,64,5,32,relu,adam,20.625084,90.534389,22.447684,1.730158,90.261459,6.315955
1,1,64,5,64,relu,adam,21.050489,93.088388,22.813062,1.730158,92.938781,6.315955
2,1,64,5,128,relu,adam,21.377949,94.896579,23.095235,1.730158,94.686615,6.315955
3,1,64,5,256,relu,adam,22.292770,100.854909,23.829643,1.730158,100.257647,6.315955
4,1,64,10,32,relu,adam,16.387848,71.508712,18.113021,1.730158,70.777130,6.315955
...,...,...,...,...,...,...,...,...,...,...,...,...
175,3,1024,10,256,relu,adam,4.309740,20.803367,5.802035,1.730158,24.065500,6.315955
176,3,1024,20,32,relu,adam,1.628339,8.209870,2.164518,1.730158,11.941126,6.315955
177,3,1024,20,64,relu,adam,2.016661,10.348829,2.665699,1.730158,15.074347,6.315955
178,3,1024,20,128,relu,adam,2.240171,11.167940,3.060246,1.730158,15.894718,6.315955


In [39]:
grouped = record_df.groupby('layer')

grouped_dfs = []

for layer, group in grouped:
    group_name = f"group_layer_{layer}"
    group_df = pd.DataFrame(group)
    grouped_dfs.append((group_name, group_df))

grouped_dfs

[('group_layer_1',
      layer  units  epochs  batch_size activation optimizer  train_mae   
  0       1     64       5          32       relu      adam  20.625084  \
  1       1     64       5          64       relu      adam  21.050489   
  2       1     64       5         128       relu      adam  21.377949   
  3       1     64       5         256       relu      adam  22.292770   
  4       1     64      10          32       relu      adam  16.387848   
  5       1     64      10          64       relu      adam  19.180447   
  6       1     64      10         128       relu      adam  20.583891   
  7       1     64      10         256       relu      adam  22.323721   
  8       1     64      20          32       relu      adam   8.392783   
  9       1     64      20          64       relu      adam  15.503694   
  10      1     64      20         128       relu      adam  19.001104   
  11      1     64      20         256       relu      adam  20.262039   
  12      1    128 

In [48]:
group_name = 'group_layer_1'
group_df = next(df for name, df in grouped_dfs if name == group_name)
# print(group_df)
sorted_group_df = group_df[['layer', 'units', 'epochs', 'batch_size', 'train_mae', 'train_mape', 'train_rmse', 'test_mae', 'test_mape', 'test_rmse']]


sorted_group_df = sorted_group_df.sort_values(by=['test_mae', 'test_mape', 'test_rmse'], ascending=[True, True, True])
sorted_group_df

,layer,units,epochs,batch_size,train_mae,train_mape,train_rmse,test_mae,test_mape,test_rmse
56,1,1024,20,32,2.964624,14.927915,4.096106,1.730158,20.489460,6.315955
44,1,512,20,32,3.362607,17.193228,4.620353,1.730158,23.345284,6.315955
57,1,1024,20,64,3.528908,17.951702,4.941872,1.730158,23.992728,6.315955
52,1,1024,10,32,3.774170,19.411805,5.267811,1.730158,25.882888,6.315955
32,1,256,20,32,3.781900,19.497475,5.305309,1.730158,25.943652,6.315955
45,1,512,20,64,4.601748,23.102295,6.185968,1.730158,28.998759,6.315955
20,1,128,20,32,5.214283,25.511745,6.876645,1.730158,29.981712,6.315955
40,1,512,10,32,5.313101,26.113787,7.072953,1.730158,31.729218,6.315955
58,1,1024,20,128,6.181656,30.087179,8.000559,1.730158,34.970397,6.315955
33,1,256,20,64,7.055721,32.721409,9.059544,1.730158,37.716699,6.315955
